In [1]:
%load_ext autoreload
%autoreload 2

# Simulating the effect of mutations on sRNA networks

In this notebook we will demonstrate a workflow for simulating mutations in RNA and examining the effect that this would have on RNA networks / circuits. We will use the `synbio_morpher` to predict the interaction strength and simulate the effect on RNA.

## Imports

In [2]:
import os
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import pandas as pd

from synbio_morpher.srv.io.manage.script_manager import script_preamble
from synbio_morpher.utils.common.setup import construct_circuit_from_cfg, prepare_config, expand_config
from synbio_morpher.utils.evolution.evolver import Evolver
from synbio_morpher.utils.circuit.agnostic_circuits.circuit_manager import CircuitModeller

import numpy as np
import jax
jax.config.update('jax_platform_name', 'gpu')



## Introduction to `synbio_morpher`

The package `synbio_morpher` allows for easy construction of genetic circuits and their simulation. A genetic circuit can be created by putting all the component RNA species into a FASTA file, or by directly creating a dictionary that contains all the RNA component parts. A configuration file governs customisations involved in specifying the creation, mutation, and simulation of circuits, each of which are created as a `Circuit` object. The `CircuitModller` class can then be used on batched `Circuit` objects and simulate the time series of these circuits and their mutated counterparts in parallel.

### First let's make a FASTA for the intended toy circuit

In [3]:
fn = os.path.join('..', 'data', 'sRNA', 'RNAInter', 'Download_data_RR.csv')
data = pd.read_csv(fn)
try:
    data = data.drop(columns=['Unnamed: 0'])
except:
    pass

In [4]:
filt = ((~ data['Sequence1'].isna()) & (~ data['Sequence2'].isna()))
data[filt]['Interactor1.Symbol'].value_counts()[data['Interactor1.Symbol'].value_counts() == 4]

Interactor1.Symbol
galK    4
ompF    4
flhD    4
Name: count, dtype: int64

In [18]:
sRNA1 = 'galK'

filt = (data['Interactor1.Symbol'] == sRNA1) & ((~ data['Sequence1'].isna()) & (~ data['Sequence2'].isna()))
data[filt].head()

,index,RNAInterID,Interactor1.Symbol,Category1,Species1,Interactor2.Symbol,Category2,Species2,Raw_ID1,Raw_ID2,score,strong,weak,predict,Sequence1,Sequence2
110,1713510,RR05458723,galK,mRNA,Escherichia coli str. K-12 substr. MG1655,micA,sRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:945358,NCBI:2847697,0.1983,Northern blot,NaN,NaN,ACTTACTACTGTGCCTTTCCTAGGCCTTACTGGGGGATCACCACCT...,CAAAAATGGTGTCGCTCGTATTATTGTCGAGAAACCTTTCGGCAAG...
111,1713511,RR05458724,galK,mRNA,Escherichia coli str. K-12 substr. MG1655,micC,sRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:945358,NCBI:2847713,0.1983,Northern blot,NaN,NaN,ACTTACTACTGTGCCTTTCCTAGGCCTTACTGGGGGATCACCACCT...,ATTTACGCTCATTGCGCATGGGTGTTTGTCCAGCAATTCTTGAATC...
112,1713512,RR05458725,galK,mRNA,Escherichia coli str. K-12 substr. MG1655,micF,sRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:945358,NCBI:2847742,0.1983,Northern blot,NaN,NaN,ACTTACTACTGTGCCTTTCCTAGGCCTTACTGGGGGATCACCACCT...,AAAGCCAACTCAACTCAACTCATCAAAATGCGCTTCATCGTCTCTC...
113,1713513,RR05458726,galK,mRNA,Escherichia coli str. K-12 substr. MG1655,ryhB,sRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:945358,NCBI:2847761,0.1983,Northern blot,NaN,NaN,ACTTACTACTGTGCCTTTCCTAGGCCTTACTGGGGGATCACCACCT...,TTTGAGAGCGTCTCTGTCCCTCGTTTTGCGGTTAAGCCGCATCCAT...


Turn everything into RNA

In [19]:
data['Sequence1'] = data['Sequence1'].str.replace('T', 'U')
data['Sequence2'] = data['Sequence2'].str.replace('T', 'U')

In [20]:
circuit_fasta = 'data/circuit.fasta'
circuit_fasta_d = {}

if not os.path.exists('data'):
    os.mkdir('data')
    

with open(circuit_fasta, 'w') as f:

    for interactor, seqN in [('Interactor1.Symbol', 'Sequence1'), ('Interactor2.Symbol', 'Sequence2')]:
        for r in list(data[filt][interactor].unique()):
            circuit_fasta_d[r] = data[data[interactor] == r][seqN].iloc[0]
            f.write('>' + r + '\n')
            f.write(data[data[interactor] == r][seqN].iloc[0])
            f.write('\n')

In [21]:
print('Length of each RNA molecule is the following:')
{k: len(i) for k, i in circuit_fasta_d.items()}

Length of each RNA molecule is the following:


{'galK': 57, 'micA': 447, 'micC': 166, 'micF': 488, 'ryhB': 490}

### Configuration for simulation

In [22]:
config = {
    "experiment": {
        "purpose": "tests",
        "no_visualisations": False,
        "no_numerical": False,
        "debug_mode": False
    },
    "data_path": circuit_fasta,
    "system_type": "RNA",
    "include_prod_deg": False,
    "interaction_simulator": {
        "name": "IntaRNA",
        "postprocess": True
    },
    "mutations_args": {
        "algorithm": "random",
        "mutation_counts": 0,
        "mutation_nums_within_sequence": [1, 3, 5, 20],
        "mutation_nums_per_position": 1,
        "concurrent_species_to_mutate": "single_species_at_a_time",
        "seed": 0
    },
    "signal": {
        "inputs": [sRNA1],
        "outputs": [],
        "function_name": "step_function",
        "function_kwargs": {
            "impulse_center": 1,
            "impulse_halfwidth": 5,
            "target": 2
        }
    },
    "simulation": {
        "dt0": 0.1,
        "t0": 0,
        "t1": 100,
        "tmax": 2000,
        "solver": "diffrax",
        "use_batch_mutations": True,
        "interaction_factor": 1,
        "batch_size": 20000,
        "max_circuits": 60000,
        "device": "gpu",
        "threshold_steady_states": 0.05,
        "use_rate_scaling": True
    },
    "molecular_params_factor": 1,
    "molecular_params": {
        "avg_mRNA_per_cell": 100,
        "cell_doubling_time": 1200,
        "creation_rate": 2.35,
        "starting_copynumbers": 200,
        "degradation_rate": 0.01175,
        "association_binding_rate": 1000000
    }
}

### Circuit construction

In [23]:
data_writer=None
config, data_writer = script_preamble(config, data_writer)
config_file = expand_config(config=config)
config_file = prepare_config(config_file)
circuit = construct_circuit_from_cfg(prev_configs=None, config_file=config_file)

circuit = Evolver(data_writer=data_writer, sequence_type=config_file.get('system_type'), 
                seed=config_file.get('mutations_args', {}).get('seed', np.random.randint(1000))).mutate(
                    circuit,
                    write_to_subsystem=True, algorithm=config_file.get('mutations_args', {}).get('algorithm', 'random'))

In [24]:
circuit.model.species

[Species: ('galK', 'galK'),
 Species: ('galK', 'micA'),
 Species: ('galK', 'micC'),
 Species: ('galK', 'micF'),
 Species: ('galK', 'ryhB'),
 Species: ('micA', 'micA'),
 Species: ('micA', 'micC'),
 Species: ('micA', 'micF'),
 Species: ('micA', 'ryhB'),
 Species: ('micC', 'micC'),
 Species: ('micC', 'micF'),
 Species: ('micC', 'ryhB'),
 Species: ('micF', 'micF'),
 Species: ('micF', 'ryhB'),
 Species: ('ryhB', 'ryhB'),
 Species: galK,
 Species: micA,
 Species: micC,
 Species: micF,
 Species: ryhB]

In [25]:
circuit.model.reactions

[Reaction(input=[Species: galK, Species: galK], output=[Species: ('galK', 'galK')], forward_rate=nan, reverse_rate=nan),
 Reaction(input=[Species: galK, Species: micA], output=[Species: ('galK', 'micA')], forward_rate=nan, reverse_rate=nan),
 Reaction(input=[Species: galK, Species: micC], output=[Species: ('galK', 'micC')], forward_rate=nan, reverse_rate=nan),
 Reaction(input=[Species: galK, Species: micF], output=[Species: ('galK', 'micF')], forward_rate=nan, reverse_rate=nan),
 Reaction(input=[Species: galK, Species: ryhB], output=[Species: ('galK', 'ryhB')], forward_rate=nan, reverse_rate=nan),
 Reaction(input=[Species: micA, Species: micA], output=[Species: ('micA', 'micA')], forward_rate=nan, reverse_rate=nan),
 Reaction(input=[Species: micA, Species: micC], output=[Species: ('micA', 'micC')], forward_rate=nan, reverse_rate=nan),
 Reaction(input=[Species: micA, Species: micF], output=[Species: ('micA', 'micF')], forward_rate=nan, reverse_rate=nan),
 Reaction(input=[Species: micA, 

In [26]:
modeller = CircuitModeller(result_writer=data_writer, config=config_file)
circuits = modeller.batch_circuits(
    circuits=[circuit],
    write_to_subsystem=True, batch_size=config_file['simulation'].get('batch_size', 100),
    methods={
        "compute_interactions": {},
        "init_circuits": {'batch': True},
        "simulate_signal_batch": {'ref_circuit': None,
                                  'batch': True},
        "write_results": {'no_visualisations': config_file['experiment'].get('no_visualisations', True),
                          'no_numerical': config_file['experiment'].get('no_numerical', False)}
    })

circuit_manager.py:batch_circuits():600: 	From 1 circuits, a total of 1 mutated circuits will be simulated. WARNING
circuit_manager.py:batch_circuits():607: 		Starting new round of viable circuits (0 - 1 / 1) WARNING
circuit_manager.py:batch_circuits():625: 		Making subcircuits 0 - 1 took 2.1e-05s WARNING
circuit_manager.py:batch_circuits():631: 	Batching 0 - 20000 circuits (out of 0 - 1 (total: 1)) (Circuits: 0 - 1 of 1) WARNING
circuit_manager.py:run_batch():662: 		Running 1 Subcircuits - circuit: compute_interactions WARNING


circuit_manager.py:run_batch():698: 		Method compute_interactions took 39.389068s WARNING
circuit_manager.py:run_batch():662: 		Running 1 Subcircuits - circuit: init_circuits WARNING


Steady states:  100  iterations.  5  left to steady out.  0:01:11.083432


circuit_manager.py:run_batch():698: 		Method init_circuits took 143.772797s WARNING
circuit_manager.py:run_batch():662: 		Running 1 Subcircuits - circuit: simulate_signal_batch WARNING


Done:  0:02:23.739697


circuit_manager.py:simulate_signal_batch():343: 		Simulating signal took 72.98653s WARNING


Done:  0:01:12.947471


circuit_manager.py:simulate_signal_batch():401: 		Calculating analytics took 1.340157s WARNING
circuit_manager.py:run_batch():698: 		Method simulate_signal_batch took 74.4987s WARNING
circuit_manager.py:run_batch():662: 		Running 1 Subcircuits - circuit: write_results WARNING
circuit_manager.py:run_batch():698: 		Method write_results took 0.41583s WARNING
circuit_manager.py:batch_circuits():645: Single batch: 0:04:18.080790 
Projected time: 258.08079s 
Total time: 0:04:18.080808 WARNING
